In [ ]:
import jams
import os
import matplotlib.pyplot as plt
#este modulo es local (interpreter.py) y contiene las funciones que se usan para visualizar los jams
import interpreter as itp
## Change This Cell!
gs_path = "./data/"
anno_dir = "annotation/"
excerpt = "04_BN2-131-B_solo"

: 

In [ ]:
jam = jams.load(os.path.join(gs_path, anno_dir, excerpt + ".jams"))

: 

In [ ]:
jam.search(namespace='chord')

: 

In [2]:
from mido import MidiFile, MidiTrack, Message, MetaMessage
import os
import jams
import mido

def write_midi(inp_file, output_file=None):
    #si ya es .midi, no haga nada
    _, ext = os.path.splitext(inp_file)
    if ext == '.midi':
        return
    jam = jams.load(inp_file) 
    notes = jam.search(namespace='note_midi')
    bpm = jam.search(namespace='tempo')[0].data[0].value
    all_notes = [list(note.data) for note in notes]
    strings = [[int(note.annotation_metadata.data_source)+1]*len(note.data) for note in notes]
    strings = [string for sublist in strings for string in sublist]
    all_notes = [note for sublist in all_notes for note in sublist]
    all_notes_with_strings = list(zip(all_notes, strings))
    strings = sorted(all_notes_with_strings, key=lambda x: x[0].time)
    mid = MidiFile()
    tempo = mido.bpm2tempo(bpm)
    ticks_per_beat = mid.ticks_per_beat
    tracks = []
    for string in range(6):
        track = MidiTrack()
        mid.tracks.append(track)
        # Set tempo
        track.append(MetaMessage('set_tempo', tempo=tempo, time=0))
        # Set instrument to Guitar (program number 26)
        track.append(Message('program_change', program=26, time=0))
        track.append(MetaMessage('time_signature', numerator=4, denominator=4, time=0))
        tracks.append(track)

    last_tick = [0] * 6  # tick de cada cuerda

    for note in strings:
        duration = note[0].duration
        start_time = note[0].time
        pitch_freq = note[0].value
        midi_note = int(pitch_freq)
        
        start_tick = mido.second2tick(start_time, ticks_per_beat, tempo)
        duration_ticks = mido.second2tick(duration, ticks_per_beat, tempo)
        
        string = note[1] - 1
        delta_time = max(0, int(start_tick - last_tick[string]))  # Ensure non-negative delta_time

        #time not absolute, relative u.u 
        tracks[string].append(Message('note_on', note=midi_note, velocity=64, time=delta_time))
        tracks[string].append(Message('note_off', note=midi_note, velocity=64, time=duration_ticks))
        
        last_tick[string] = start_tick + duration_ticks #el ultimo evento fue en este tick
    if output_file:
        mid.save(output_file)
    else:
        mid.save(inp_file.replace('.jams', '.mid'))





In [5]:
for raiz, dirs, archivos in os.walk('./data/annotation'):
    for archivo in archivos:
        ruta_completa = os.path.join(raiz, archivo)
        print(ruta_completa)
        try:
            name_ = archivo.replace('.jams','.mid')
            write_midi(ruta_completa,f'data/transformed/{name_}')
        except Exception as e:
            print(f"Error al convertir el archivo {ruta_completa}: {e}")
            continue

./data/annotation/01_SS3-84-Bb_solo.jams
./data/annotation/04_Rock3-117-Bb_solo.jams
./data/annotation/03_Jazz3-137-Eb_comp.jams
./data/annotation/00_Rock1-90-C#_solo.jams
./data/annotation/00_Rock2-142-D_solo.jams
./data/annotation/00_Jazz3-137-Eb_solo.jams
./data/annotation/01_Jazz1-200-B_comp.jams
./data/annotation/01_Rock3-117-Bb_comp.jams
./data/annotation/02_BN1-147-Gb_comp.jams
./data/annotation/00_Jazz2-187-F#_solo.jams
./data/annotation/04_Jazz2-110-Bb_solo.jams
./data/annotation/04_Rock1-130-A_solo.jams
./data/annotation/01_Funk2-119-G_solo.jams
./data/annotation/04_BN2-131-B_solo.jams
./data/annotation/03_Rock2-85-F_comp.jams
./data/annotation/05_BN2-131-B_solo.jams
./data/annotation/05_Rock2-85-F_solo.jams
./data/annotation/04_Jazz2-187-F#_comp.jams
./data/annotation/03_Jazz2-187-F#_solo.jams
./data/annotation/03_Funk2-119-G_solo.jams
./data/annotation/04_Rock1-90-C#_comp.jams
./data/annotation/05_Funk3-112-C#_solo.jams
./data/annotation/03_Funk3-98-A_comp.jams
./data/annot

In [ ]:
from mido import MidiFile, MidiTrack, Message, MetaMessage
import jams
import mido

def write_midi(inp_file, output_file=None):
    jam = jams.load(inp_file) 
    notes = jam.search(namespace='note_midi')
    bpm = jam.search(namespace='tempo')[0].data[0].value
    all_notes = [list(note.data) for note in notes]
    strings = [[int(note.annotation_metadata.data_source)+1]*len(note.data) for note in notes]
    strings = [string for sublist in strings for string in sublist]
    all_notes = [note for sublist in all_notes for note in sublist]
    all_notes_with_strings = list(zip(all_notes, strings))
    strings = sorted(all_notes_with_strings, key=lambda x: x[0].time)
    mid = MidiFile()
    tempo = mido.bpm2tempo(bpm)
    ticks_per_beat = mid.ticks_per_beat

    tracks = []
    for string in range(6):
        track = MidiTrack()
        mid.tracks.append(track)
        # Set tempo
        track.append(MetaMessage('set_tempo', tempo=tempo, time=0))
        # Set instrument to Guitar (program number 26)
        track.append(Message('program_change', program=26, time=0))
        track.append(MetaMessage('time_signature', numerator=4, denominator=4, time=0))
        tracks.append(track)

    last_tick = [0] * 6  # tick de cada cuerda

    for note in strings:
        duration = note[0].duration
        start_time = note[0].time
        pitch_freq = note[0].value
        midi_note = int(pitch_freq)
        
        start_tick = mido.second2tick(start_time, ticks_per_beat, tempo)
        duration_ticks = mido.second2tick(duration, ticks_per_beat, tempo)
        
        string = note[1] - 1
        delta_time = max(0, int(start_tick - last_tick[string]))  # Ensure non-negative delta_time

        if start_tick < last_tick[string]:
            # Overlapping note, send note_off for the previous note first
            overlap_time = max(0, last_tick[string] - start_tick)
            tracks[string].append(Message('note_off', note=midi_note, velocity=64, time=overlap_time))
            delta_time = 0  # No additional delta time needed for the note_on message

        tracks[string].append(Message('note_on', note=midi_note, velocity=64, time=delta_time))
        tracks[string].append(Message('note_off', note=midi_note, velocity=64, time=duration_ticks))
        
        last_tick[string] = max(start_tick + duration_ticks, last_tick[string] + duration_ticks) # Update last tick to the maximum of the overlapping notes

        print(f'Note: {midi_note}, Start time (s): {start_time}, Duration (s): {duration}, Start tick: {start_tick}, Duration ticks: {duration_ticks}, String: {string + 1}')

    if output_file:
        mid.save(output_file)
    else:
        mid.save(inp_file.replace('.jams', '.mid'))

write_midi('./data/annotation/04_BN2-131-B_solo.jams','./04_BN2-131-B_solo.mid')


: 

In [ ]:
#2. midi a music21 object para añadirle chords y luego guardar como musicXML 
import music21

# Specify the path to your MIDI file
midi_path =  "./04_BN2-131-B_solo.mid"
mf = music21.midi.MidiFile()
mf.open(midi_path)
mf.read()

# Convert MIDI file to music21 stream
stream = music21.midi.translate.midiFileToStream(mf)
stream.insert(0, music21.metadata.Metadata())
stream.metadata.title = "04_BN2-131-B_solo"

# Add chords from jam
chords = jam.search(namespace='chord')[0].data
last_time = 0
for chord in chords:
    chord_name = chord.value.replace(':', '').replace('hdim7','m7b5')
    print(chord_name)
    bpm = jam.search(namespace='tempo')[0].data[0].value
    chord_time = chord.time
    chords_offset = (chord_time - last_time)*bpm/60
    
    #add silence if needed
    if chords_offset > 0:
        rest = music21.note.Rest()
        rest.duration = music21.duration.Duration(round(chords_offset,4))
        stream.insert(chord_time, rest)
    # add chord
    chord_duration = chord.duration
    chord_duration_beats = round(bpm/60 * chord_duration,4)
    chord_obj = music21.harmony.ChordSymbol(chord_name)
    chord_obj.quarterLength = chord_duration_beats
    # set last note 
    last_time = chord_time + chord_duration
    print(chord_duration_beats)
    
    #insert chord
    stream.insert(chord_time, chord_obj)

# Save as MusicXML
stream.write('musicxml', f'./data/04_BN2-131-B_solo.xml')

: 

In [ ]:
import music21

# Specify the path to your MIDI file
midi_path = "./04_BN2-131-B_solo.mid"
mf = music21.midi.MidiFile()
mf.open(midi_path)
mf.read()
mf.close()

# Convert MIDI file to music21 stream
midi_stream = music21.midi.translate.midiFileToStream(mf)
midi_stream.insert(0, music21.metadata.Metadata())
midi_stream.metadata.title = "04_BN2-131-B_solo"
Streams = midi_stream.getElementsByClass('Part')
Streams[0].getElementsByClass('Measure')[0].show()

# # Add chords from jam
# chords = jam.search(namespace='chord')[0].data
# tempo =jam.search(namespace='tempo')[0].data[0].value

# Measures = Streams[0].getElementsByClass('Measure')


# for chord in chords:
#     chord_name = chord.value.replace(':', '').replace('hdim7', 'm7b5')
#     print(chord_name)
#     chord_duration = chord.duration
#     chord_duration_beats = round(tempo / 60 * chord_duration, 4)
#     num_measure = int(((chords[-1].time)*bpm/60)/4)-1
#     # Add chord symbol
#     chord_obj = music21.harmony.ChordSymbol(chord_name)
#     chord_obj.quarterLength = chord_duration_beats
#     Measures[num_measure].insert(0, chord_obj)




# # Save as MusicXML
# midi_stream.write('musicxml', './data/04_BN2-131-B_solo.xml')


: 

In [24]:
import music21
#this one works
# Specify the path to your MIDI file
def midi2xml_withchords(file_name,output_file):
    mf = music21.midi.MidiFile()
    mf.open('./data/transformed/'+file_name)
    mf.read()
    mf.close()
    #si ya es .xml, no haga nada
    _, ext = os.path.splitext(file_name)
    if ext == '.xml':
        return
    path_to_jams = './data/annotation/'
    corresponding_jam = file_name.replace('.mid','.jams')
    jam = jams.load(path_to_jams+corresponding_jam) 
    # Convert MIDI file to music21 stream
    midi_stream = music21.midi.translate.midiFileToStream(mf)

    # Assuming the existence of some `jam` object or data that you fetch elsewhere
    # For this example, let's create some dummy chord data to work with
    # Example chord data and tempo - you should replace this with your actual data
    chords = jam.search(namespace='chord')[0].data
    tempo = jam.search(namespace='tempo')[0].data[0].value # bpm, this should come from your `jam` object

    # You might want to process the first part only if there are multiple parts
    part = midi_stream.getElementsByClass('Part')
    max_part = max(part, key=lambda part: len(part.getElementsByClass('Measure')))
    # Process each chord
    for chord_data in chords:
        chord_name = chord_data.value.replace(':', '').replace('hdim7', 'm7b5')
        # print(chord_name)
        chord_duration = chord_data.duration
        # Calculate the duration in quarter lengths assuming 60 seconds per minute
        chord_duration_quarters = (tempo / 60) * chord_duration
        # Calculate where to insert the chord in terms of measures
        start_measure = int((chord_data.time * tempo / 60) / 4)  # Simplified estimate
        # print(start_measure)
        chord_obj = music21.harmony.ChordSymbol(chord_name)
        chord_obj.duration.quarterLength = chord_duration_quarters
        max_part.getElementsByClass('Measure')[start_measure].insert(0, chord_obj)  # measures are 1-indexed

    # Set metadata
    midi_stream.metadata = music21.metadata.Metadata()
    midi_stream.metadata.title = "04_BN2-131-B_solo"

    # Save as MusicXML (for viewing in notation software)
    midi_stream.write('musicxml', '04_BN2-131-B_solo.xml')

    if output_file:
        midi_stream.write('musicxml',output_file)
    else:
        midi_stream.write('musicxml',inp_file.replace('.mid', '.xml'))




In [23]:
for raiz, dirs, archivos in os.walk('./data/transformed/'):
    for archivo in archivos:
        ruta_completa = os.path.join(raiz, archivo)
        print(ruta_completa)
        try:
            name_ = archivo.replace('.mid','.xml')
            midi2xml_withchords(archivo,f'data/xmls/{name_}')
        except Exception as e:
            print(f"Error al convertir el archivo {ruta_completa}: {e}")
            continue

./data/transformed/03_Rock2-142-D_solo.mid
Gmin
0
C7
1
Fmaj
2
A#maj
3
Em7b5
4
A7
5
Dmin
6
Gmin
8
C7
9
Fmaj
10
A#maj
11
Em7b5
12
A7
13
Dmin
14
./data/transformed/01_Funk2-108-Eb_solo.mid
G#min
0
C#7
1
F#maj
2
Bmaj
3
Fm7b5
4
A#7
5
D#min
6
G#min
8
C#7
9
F#maj
10
Bmaj
11
Fm7b5
12
A#7
13
D#min
14
./data/transformed/03_Rock1-130-A_solo.mid
Amaj
0
Dmaj
4
Amaj
6
Emaj
8
Dmaj
9
Amaj
10


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f7555e0adc0>>
Traceback (most recent call last):
  File "/home/acmoral/miniconda3/envs/projfinalml/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


./data/transformed/01_Funk2-119-G_comp.mid
Cmin
0
F7
1
A#maj
2
D#maj
3
Am7b5
4
D7
5
Gmin
6
Cmin
8
F7
9
A#maj
10
D#maj
11
Am7b5
12
D7
13
Gmin
14
./data/transformed/05_Rock3-117-Bb_comp.mid
A#maj
0
Fmaj
0
Gmin
1
Dmin
2
D#maj
3
A#maj
5
D#maj
5
Fmaj
6
A#maj
7
Fmaj
9
Gmin
10
Dmin
10
D#maj
11
A#maj
12
D#maj
13
Fmaj
14
./data/transformed/01_SS3-84-Bb_solo.mid
A#maj
0
Fmaj
1
Gmin
2
Dmin
3
D#maj
4
A#maj
5
D#maj
6
Fmaj
7
A#maj
8
Fmaj
9
Gmin
10
Dmin
11
D#maj
12
A#maj
13
D#maj
14
Fmaj
15
./data/transformed/05_Funk2-108-Eb_comp.mid
G#min
0
C#7
1
F#maj
2
Bmaj
3
Fm7b5
4
A#7
5
D#min
6
G#min
8
C#7
9
F#maj
10
Bmaj
11
Fm7b5
12
A#7
13
D#min
14
./data/transformed/01_BN2-166-Ab_solo.mid
C#min
0
F#7
1
Bmaj
2
Emaj
3
A#m7b5
4
D#7
5
G#min
6
C#min
8
F#7
9
Bmaj
10
Emaj
11
A#m7b5
12
D#7
13
G#min
14
./data/transformed/05_Rock3-148-C_comp.mid
Cmaj
0
Gmaj
1
Amin
2
Emin
3
Fmaj
4
Cmaj
5
Fmaj
6
Gmaj
7
Cmaj
8
Gmaj
9
Amin
10
Emin
11
Fmaj
12
Cmaj
13
Fmaj
14
Gmaj
15
./data/transformed/03_Funk2-119-G_comp.mid
Cmin
0
F7
1
A#m

In [ ]:
from music21 import stream, note, harmony, meter

# Create a new stream
my_stream = stream.Stream()

# Create a measure with a time signature
my_measure = stream.Measure()
my_measure.timeSignature = meter.TimeSignature('4/4')

# Define some notes
note1 = note.Note('C4')  # Middle C
note1.duration.quarterLength = 2  # Half note
note2 = note.Note('E4')  # E above middle C
note2.duration.quarterLength = 2  # Half note
note3 = note.Note('G4')  # G above middle C
note3.duration.quarterLength = 2 # Half note
note4 = note.Note('B4')  # B above middle C
note4.duration.quarterLength = 2 # Half note

# Create a chord symbol
chord_obj = harmony.ChordSymbol('C')
chord_obj.quarterLength = 2
chord_obj_2 = harmony.ChordSymbol('E')
chord_obj_2.quarterLength = 2
# Add the chord symbol to the measure (on the first beat)
my_measure.insert(0, chord_obj)
my_measure.insert(2, chord_obj_2)

second_measure = stream.Measure()
second_measure.timeSignature = meter.TimeSignature('4/4')

second_measure.append(note3)
second_measure.append(note4)
#add chords
chord_obj_3 = harmony.ChordSymbol('G')
chord_obj_3.quarterLength = 2
chord_obj_4 = harmony.ChordSymbol('B')
chord_obj_4.quarterLength = 2
second_measure.insert(0, chord_obj_3)
second_measure.insert(2, chord_obj_4)

# Add the notes to the measure
my_measure.append(note1)
my_measure.append(note2)

# Add the measure to the stream
my_stream.append(my_measure)
my_stream.append(second_measure)

# Show the stream in textual format
my_stream.show('text')

# Optionally, display the stream in musical notation (requires MuseScore or similar software)
my_stream.show()



: 